In [37]:
import sqlite3
import numpy as np
import pandas as pd
import pickle
from collections import Counter

datarange = ['2015-01']
for timeframe in datarange:
    connection = sqlite3.connect('./chatdata/{}.db'.format(timeframe))
    c = connection.cursor()
    limit = 5000
    last_unix = 0
    cur_length = limit
    counter = 0
    test_done = False
    while cur_length == limit :
        dataframe = pd.read_sql("SELECT * from parent_reply where unix > {} AND parent NOT NULL AND score > 2 ORDER BY unix ASC LIMIT {}".format(last_unix,limit),connection)
        last_unix = dataframe.tail(1)['unix'].values[0]
        cur_length =  len(dataframe)
        wordliststr=""
        if not test_done:
            with open("./chatdata/{}test.from".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['parent'].values:
                    f.write(content+'\n')
                    wordliststr +=content

            with open("./chatdata/{}test.to".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['comment'].values:
                    f.write(content+'\n')
                    wordliststr +=content

            #for creating a test file in which parent and reply are in sequential order      
            comList = dataframe['parent'].values
            repList = dataframe['comment'].values
            logdict={}
            for i in range(len(comList)):
                logdict[comList[i]] = repList[i]
            np.save( "./chatdata/comrepTest{}.npy".format(timeframe), logdict)
            
            wordlistdict = Counter(wordliststr.split())
            wordlist = list(wordlistdict.keys())
            with open("wordlist.txt","wb")as fp:
                pickle.dump(wordlist,fp)
            
            test_done = True
            
        else:
            # training files in which from contains the comment and to contain the correspinding reply at corresponding line number
            with open("./chatdata/{}train.from".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['parent'].values:
                    f.write(content+'\n')
                    wordliststr += content

            with open("./chatdata/{}train.to".format(timeframe),'a',encoding='utf8') as f:
                for content in dataframe['comment'].values:
                    f.write(content+'\n')
                    wordliststr += content

              #for creating a training file in which parent and reply are in sequential order      
            comList = dataframe['parent'].values
            repList = dataframe['comment'].values
            logdict={}
            for i in range(len(comList)):
                logdict[comList[i]] = repList[i]
            np.save( "./chatdata/comrepTrain{}.npy".format(timeframe), logdict)
            
            wordlistdict = Counter(wordliststr.split())
            wordlist = list(wordlistdict.keys())
            with open("wordlist.txt","wb")as fp:
                pickle.dump(wordlist,fp)
            test_done = True 

        
        counter+=1
        
        if counter%20 == 0 :
            print(counter*limit,'rows completed so far')
            
            
        
        
        
        
        

17
